In [1]:
import os
import time
import pickle
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import log_loss, roc_auc_score

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss, MSELoss
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F
import optuna
from sklearn.model_selection import StratifiedKFold



train = pd.read_csv('/home/yihang_toby/data/all_features.csv')


In [2]:
importance = pd.read_csv('/home/yihang_toby/model/xgboost/feature_importance.csv',index_col= 0)
feat_cols = list(importance.sort_values(by = '0',ascending = False)[:200].index)

In [12]:
all_feat_cols = feat_cols

In [6]:
train[feat_cols].mean().to_csv('f_mean.csv')

In [8]:
!pwd

/home/yihang_toby/torch


In [9]:
(train['resp']>0).mean()

0.504069666022587

In [13]:
class Model(nn.Module):
    def __init__(self,hidden_size,dropout_rate,activate):
        super(Model, self).__init__()
        self.batch_norm0 = nn.BatchNorm1d(len(all_feat_cols))
        self.dropout0 = nn.Dropout(0.2)
        self.dense1 = nn.Linear(len(all_feat_cols), hidden_size)
        self.batch_norm1 = nn.BatchNorm1d(hidden_size)
        self.dropout1 = nn.Dropout(dropout_rate/10)

        self.dense2 = nn.Linear(hidden_size+len(all_feat_cols), hidden_size)
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(dropout_rate/10)

        self.dense3 = nn.Linear(hidden_size+hidden_size, hidden_size)
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(dropout_rate/10)

        self.dense4 = nn.Linear(hidden_size+hidden_size, hidden_size)
        self.batch_norm4 = nn.BatchNorm1d(hidden_size)
        self.dropout4 = nn.Dropout(dropout_rate/10)

        self.dense5 = nn.Linear(hidden_size+hidden_size, len(target_cols))

        self.activate = activate
        self.Relu = nn.ReLU(inplace=True)
        self.PReLU = nn.PReLU()
        self.LeakyReLU = nn.LeakyReLU(negative_slope=0.01, inplace=True)
        # self.GeLU = nn.GELU()
        self.RReLU = nn.RReLU()

    def forward(self, x):
        x = self.batch_norm0(x)
        x = self.dropout0(x)

        x1 = self.dense1(x)
        x1 = self.batch_norm1(x1)
        # x = F.relu(x)
        # x = self.PReLU(x)
        exec('x1 = self.'+self.activate+'(x1)')
        x1 = self.dropout1(x1)

        x = torch.cat([x, x1], 1)

        x2 = self.dense2(x)
        x2 = self.batch_norm2(x2)
        # x = F.relu(x)
        # x = self.PReLU(x)
        exec('x2 = self.'+self.activate+'(x2)')
        x2 = self.dropout2(x2)

        x = torch.cat([x1, x2], 1)

        x3 = self.dense3(x)
        x3 = self.batch_norm3(x3)
        # x = F.relu(x)
        # x = self.PReLU(x)
        exec('x3 = self.'+self.activate+'(x3)')
        x3 = self.dropout3(x3)

        x = torch.cat([x2, x3], 1)

        x4 = self.dense4(x)
        x4 = self.batch_norm4(x4)
        # x = F.relu(x)
        # x = self.PReLU(x)
        exec('x4 = self.'+self.activate+'(x4)')
        x4 = self.dropout4(x4)

        x = torch.cat([x3, x4], 1)

        x = self.dense5(x)

        return x

In [49]:
target_cols = ['resp','resp_1','resp_2','resp_3','resp_4']

In [47]:
class MarketDataset:
    def __init__(self, df):
        self.features = df[all_feat_cols].values

        self.label = df[target_cols].values.reshape(-1, len(target_cols))

        self.weight = df['weight'].values.reshape(-1, 1)

    def __len__(self):
        return len(self.label)

    def __getitem__(self, idx):
        return {
            'features': torch.tensor(self.features[idx], dtype=torch.float),
            'label': torch.tensor(self.label[idx], dtype=torch.float),
            'weight':torch.tensor(self.weight[idx],dtype = torch.float)
        }


In [42]:
models = []

for _fold in range(5):
    device = 'cpu'
    model = Model(400,0.2,'LeakyReLU')
    model.to(device)
    model_weights = f"./train_newfeature_notweight{_fold}.pth"
    model.load_state_dict(torch.load(model_weights,map_location=torch.device('cpu')))
    models.append(model)

In [43]:
X = train[feat_cols]

In [44]:
actions = []

In [52]:
BATCH_SIZE = 8000

In [56]:
def inference_fn(model, dataloader, device):
    model.eval()
    preds = []

    for data in dataloader:
        features = data['features'].to(device)

        with torch.no_grad():
            outputs = model(features)

        preds.append(outputs.sigmoid().detach().cpu().numpy())

    preds = np.concatenate(preds).reshape(-1, len(target_cols))

    return preds

In [61]:
test_pred = np.zeros((len(train), len(target_cols)))
test_set = MarketDataset(train)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=100)
for _fold in range(2):
    torch.cuda.empty_cache()
    device = torch.device('cuda:0')
    model = Model(400,0.2,'LeakyReLU')
    model.to(device)
    model_weights = f"./train_newfeature_notweight_uscore{_fold}.pth"
    model.load_state_dict(torch.load(model_weights))

    test_pred += inference_fn(model, test_loader, device) / 2

In [68]:
for i in range(480,500):
    test_set = MarketDataset(train[i:i+1])
    test_loader = DataLoader(test_set, batch_size=1, shuffle=False, num_workers=100)
    test_pred = np.zeros((len(train), len(target_cols)))
    for _fold in range(2): 
        torch.cuda.empty_cache()
        device = torch.device('cuda:0')
        model = Model(400,0.2,'LeakyReLU')
        model.to(device)
        model_weights = f"./train_newfeature_notweight_uscore{_fold}.pth"
        model.load_state_dict(torch.load(model_weights))

        test_pred += inference_fn(model, test_loader, device) / 2
    print(test_pred)

[[0.52370986 0.52217388 0.52730095 0.52438268 0.52129334]
 [0.52370986 0.52217388 0.52730095 0.52438268 0.52129334]
 [0.52370986 0.52217388 0.52730095 0.52438268 0.52129334]
 ...
 [0.52370986 0.52217388 0.52730095 0.52438268 0.52129334]
 [0.52370986 0.52217388 0.52730095 0.52438268 0.52129334]
 [0.52370986 0.52217388 0.52730095 0.52438268 0.52129334]]
[[0.53355592 0.48335709 0.47453764 0.510392   0.55275992]
 [0.53355592 0.48335709 0.47453764 0.510392   0.55275992]
 [0.53355592 0.48335709 0.47453764 0.510392   0.55275992]
 ...
 [0.53355592 0.48335709 0.47453764 0.510392   0.55275992]
 [0.53355592 0.48335709 0.47453764 0.510392   0.55275992]
 [0.53355592 0.48335709 0.47453764 0.510392   0.55275992]]
[[0.53400351 0.50008081 0.50281575 0.51355971 0.54009853]
 [0.53400351 0.50008081 0.50281575 0.51355971 0.54009853]
 [0.53400351 0.50008081 0.50281575 0.51355971 0.54009853]
 ...
 [0.53400351 0.50008081 0.50281575 0.51355971 0.54009853]
 [0.53400351 0.50008081 0.50281575 0.51355971 0.5400985

In [66]:
test_pred[480:500]

array([[0.52370983, 0.52217382, 0.52730095, 0.52438268, 0.52129334],
       [0.5335559 , 0.48335706, 0.4745376 , 0.51039194, 0.55275992],
       [0.53400354, 0.50008081, 0.50281577, 0.51355971, 0.54009859],
       [0.4903944 , 0.44046307, 0.42460571, 0.44275691, 0.50931814],
       [0.49191935, 0.44201224, 0.42669328, 0.44556814, 0.51066181],
       [       nan,        nan,        nan,        nan,        nan],
       [       nan,        nan,        nan,        nan,        nan],
       [0.51807314, 0.481424  , 0.47038546, 0.49600521, 0.53482234],
       [0.56158814, 0.52315703, 0.53388378, 0.54747206, 0.56198189],
       [0.48748422, 0.49023055, 0.49728276, 0.48562118, 0.47887769],
       [       nan,        nan,        nan,        nan,        nan],
       [0.4452424 , 0.47755174, 0.46429648, 0.44657519, 0.44382142],
       [0.8113597 , 0.53735451, 0.56239656, 0.69996083, 0.82837367],
       [       nan,        nan,        nan,        nan,        nan],
       [0.47345378, 0.47539227, 0.

In [60]:
test_pred[:100]

array([[       nan,        nan,        nan,        nan,        nan],
       [0.54541683, 0.55348092, 0.5545046 , 0.55175203, 0.5441066 ],
       [0.4979822 , 0.5106159 , 0.51587005, 0.52036893, 0.50469224],
       [0.49839479, 0.48261628, 0.48523717, 0.49193117, 0.49934795],
       [0.52310771, 0.51354869, 0.51695497, 0.53717345, 0.53275695],
       [0.47960562, 0.54963052, 0.54858252, 0.51314647, 0.47189623],
       [0.53914431, 0.5918535 , 0.58822444, 0.55441801, 0.52688372],
       [       nan,        nan,        nan,        nan,        nan],
       [0.49519993, 0.51342468, 0.51373142, 0.50352187, 0.49299106],
       [       nan,        nan,        nan,        nan,        nan],
       [       nan,        nan,        nan,        nan,        nan],
       [0.50866961, 0.51458235, 0.51505435, 0.49805598, 0.50115015],
       [       nan,        nan,        nan,        nan,        nan],
       [0.50694911, 0.52725282, 0.5257116 , 0.52597809, 0.50927612],
       [0.49051458, 0.46891861, 0.

In [ ]:
auc_score = roc_auc_score(test[target_cols].values, test_pred)
logloss_score = log_loss(test[target_cols].values, test_pred,sample_weight = test['weight'].values)

test_pred = np.median(test_pred, axis=1)
test_pred = np.where(test_pred >= 0.5, 1, 0).astype(int)
test_score = utility_score_bincount(date=test.date.values, weight=test.weight.values, resp=test.resp.values,
                                     action=test_pred)
print(f'{NFOLDS} models test score: {test_score}\tauc_score: {auc_score:.4f}\tlogloss_score:{logloss_score:.4f}')

In [45]:
for i in range(len(X)):
    x = np.array(X.iloc[i:i+1])
    X_input = torch.from_numpy(x).to(device).double()
    preds = []
    for model in models:
        model.eval()
        model.double()
        preds.append(model(X_input).detach().numpy().reshape(1,-1))
    preds = np.concatenate(preds,axis=0)
    preds = np.mean(preds,axis=0)
    test_pred = np.median(preds)
    actions.append(int(test_pred>0.5))

KeyboardInterrupt: 

In [41]:
actions

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [3]:
print(feat_cols[:10])
print(train[feat_cols[:10]].iloc[:100])

['feature_39*abs(feature_113)', 'feature_63+feature_120/abs(feature_63)', 'feature_44/abs(feature_41)', 'feature_43/abs(feature_45)', 'feature_44+feature_45*abs(feature_41)', 'feature_41+feature_43*abs(feature_43)', 'feature_45+feature_63/abs(feature_41)', 'feature_31*abs(feature_113)', 'feature_45+feature_63', 'feature_63+feature_120*abs(feature_44)']
    feature_39*abs(feature_113)  feature_63+feature_120/abs(feature_63)  \
0                     -2.117449                                     NaN   
1                      1.453864                                     NaN   
2                      3.064786                                     NaN   
3                      1.680562                                     NaN   
4                     -0.689900                                     NaN   
..                          ...                                     ...   
95                    -1.979598                                     NaN   
96                    -0.798290              